<a href="https://colab.research.google.com/github/HiyaJain22/Text2Sql/blob/main/BART_BASE_FINETUNNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BART_BASE_FINETUNNING**

In [ ]:
!pip install datasets trl bitsandbytes transformers peft

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, PeftModel
import torch

# Initialize model and tokenizer
checkpoint = "facebook/bart-base"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

def create_prompt(question, context):
    return f"""You are a SQL expert, given the Schema, generate a SQL query for the question asked
Schema: {context}
Question: {question}"""

# Test initial model
question = "How many heads of the departments are older than 56 ?"
context = "CREATE TABLE head (age INTEGER)"

# Tokenize input
inputs = tokenizer(
    create_prompt(question, context),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
).to(device)

# Generate output
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,
    temperature=0.2,
    top_p=0.9,
    do_sample=True
)

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Initial model output:", response)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Initial model output: You are a SQL expert, given the Schema, generate a SQL query for the question asked above.Schema: CREATE TABLE head (age INTEGER)Question: How many heads of the departments are older than 56 ?


In [ ]:
# Load and prepare dataset
dataset_id = "b-mc2/sql-create-context"
data = load_dataset(dataset_id, split="train")
df = data.to_pandas()

# Prepare training data
def prepare_training_features(examples):
    inputs = [create_prompt(q, c) for q, c in zip(examples["question"], examples["context"])]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    # Tokenize targets
    labels = tokenizer(
        examples["answer"],
        max_length=128,
        padding="max_length",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert dataset to features
formatted_data = Dataset.from_pandas(df)
formatted_data = formatted_data.map(
    prepare_training_features,
    batched=True,
    remove_columns=formatted_data.column_names
)



README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
# Configure LoRA
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    target_modules=["q_proj", "v_proj"]  # BART-specific attention modules
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-base-sqllm-v1",
    per_device_train_batch_size=6,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=10,
    max_steps=5000,
    fp16=True,
    predict_with_generate=True
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_data,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the final model
trainer.save_model("bart-base-sqllm-v1-final")

# Load and test the fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)
peft_model = PeftModel.from_pretrained(
    model,
    "bart-base-sqllm-v1-final",
    device_map="auto"
)
model = peft_model.merge_and_unload()



trainable params: 1,769,472 || all params: 141,189,888 || trainable%: 1.2533


<ipython-input-3-1e211b6f54d6>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,13.073000
20,11.336300
30,10.531100
40,9.565600
50,8.081900
60,6.663500
70,5.977300
80,5.447600
90,5.152500
100,5.025900


In [ ]:
# Test cases
test_cases = [
    {
        "question": "How many heads of the departments are older than 56 ?",
        "context": "CREATE TABLE head (age INTEGER)"
    },
    {
        "question": "List the name, born state and age of the heads of departments ordered by age.",
        "context": "CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)"
    }
]

for test in test_cases:
    prompt = create_prompt(test["question"], test["context"])
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=128)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nQuestion: {test['question']}")
    print(f"Generated SQL: {result}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Question: How many heads of the departments are older than 56 ?
Generated SQL: SELECT SUM(age) FROM head WHERE age > 56

Question: List the name, born state and age of the heads of departments ordered by age.
Generated SQL: SELECT name FROM head WHERE born_state = "d departments" AND age = "order"


In [ ]:
# List to store generated SQL queries
generated_sqls = []

# Test cases
test_cases = [
    {
        "question": "How many heads of the departments are older than 56 ?",
        "context": "CREATE TABLE head (age INTEGER)"
    },
    {
        "question": "List the name, born state and age of the heads of departments ordered by age.",
        "context": "CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)"
    }
]

# Iterate through the test cases and generate SQL queries
for test in test_cases:
    prompt = create_prompt(test["question"], test["context"])
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=128)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Store the generated SQL queries in the list
    generated_sqls.append(result)

# Print the list of generated SQL queries
print(generated_sqls)


['SELECT SUM(age) FROM head WHERE age > 56', 'SELECT name FROM head WHERE born_state = "d departments" AND age = "order"']


In [ ]:
model.save_pretrained("./bart-small-model")
tokenizer.save_pretrained("./bart-small-tokenizer")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./bart-small-tokenizer/tokenizer_config.json',
 './bart-small-tokenizer/special_tokens_map.json',
 './bart-small-tokenizer/vocab.json',
 './bart-small-tokenizer/merges.txt',
 './bart-small-tokenizer/added_tokens.json',
 './bart-small-tokenizer/tokenizer.json')

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import accuracy_score


In [ ]:
# Function to calculate BLEU score
def calculate_bleu_score(predictions, references):
    if not predictions or not references:
        return 0.0  # Return 0 if either list is empty
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        # Tokenize the predictions and references (split by spaces)
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        bleu_score = sentence_bleu([ref_tokens], pred_tokens)
        bleu_scores.append(bleu_score)
    return sum(bleu_scores) / len(bleu_scores)


In [ ]:
reference_sqls = [
    "SELECT COUNT(*) FROM head WHERE age > 56",
    "SELECT name, born_state, age FROM head ORDER BY age"
]

# Now, calculate BLEU score and Exact Match
bleu = calculate_bleu_score(generated_sqls, reference_sqls)
em = calculate_exact_match(generated_sqls, reference_sqls)

print(f"BLEU score: {bleu:.4f}")
print(f"Exact Match (EM): {em:.4f}")


BLEU score: 0.3536
Exact Match (EM): 0.0000


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Specify your model and tokenizer directories
model_dir = "./bart-small-model"
tokenizer_dir = "./bart-small-tokenizer"

# Load model and tokenizer from saved directories
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Push the model and tokenizer to Hugging Face Hub
model.push_to_hub("Hiyaj/bart-small-text2sql")
tokenizer.push_to_hub("Hiyaj/bart-small-text2sql")


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Hiyaj/bart-small-text2sql/commit/476930aaedb63d829f37c19334819c29235eec1e', commit_message='Upload tokenizer', commit_description='', oid='476930aaedb63d829f37c19334819c29235eec1e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Hiyaj/bart-small-text2sql', endpoint='https://huggingface.co', repo_type='model', repo_id='Hiyaj/bart-small-text2sql'), pr_revision=None, pr_num=None)